In [1]:
# Imports and Load Raw Data
import pandas as pd
import json

raw_file = 'raw_data.xlsx'
df_raw = pd.read_excel(raw_file)

print(f"Loaded raw_data.xlsx: {df_raw.shape[0]} rows, {df_raw.shape[1]} cols")
df_raw.head()

Loaded raw_data.xlsx: 15 rows, 8 cols


,id,comm_type,raw_content,source_id,ingested_at,processed_at,is_processed,subject
0,de819d74-9b7d-4b60-863c-5f7f634a6402,call,"{""id"": ""DDCD444CF2A04CA1BA108EE5"", ""title"": ""U...",32A19D011A4C41F09FE3A2FB,2025-06-12T13:29:09,2025-03-22T11:37:50,True,Profit-focused composite time-frame
1,d4746716-3350-4e7f-952f-b39a8dc430d3,call,"{""id"": ""E462A7000DCA4A688FB880B3"", ""title"": ""E...",485F87C4F6E34F3189D1E3E9,2025-03-11T19:29:43,2025-03-06T01:09:40,True,Ergonomic dedicated process improvement
2,e509a9f2-c9a0-4ad0-9489-e22825e40488,meeting,"{""id"": ""66DE26DE41BC462AAE1BA7C8"", ""title"": ""S...",B7E98770F4A542729D65AF41,2025-03-27T06:41:29,2025-01-12T01:51:05,True,Focused clear-thinking parallelism
3,77fb5908-4181-45b8-b484-f7bd4dd294c9,meeting,"{""id"": ""69D47DC9F39A4AA29B34AEFD"", ""title"": ""S...",7A00DCB707B74395B2DD552F,2025-05-06T02:23:25,2025-03-04T16:29:27,True,Re-engineered object-oriented emulation
4,6ba3fa0e-41c1-4ed6-a782-900cc6af3580,call,"{""id"": ""0D1BABD93ED04F099F2BCE2D"", ""title"": ""S...",1C96310A145B445799924A8F,2025-05-09T05:48:27,2025-05-07T15:35:37,True,Focused 24hour policy


In [2]:
# Clean raw_content and Parse JSON

def fix_json_tail(x):
    if isinstance(x, str) and x.strip().endswith('}]'):
        return x.strip()[:-2]
    return x

def safe_json_parse(x):
    try:
        if isinstance(x, str):
            x = json.loads(x)
        if isinstance(x, dict):
            return x
        return {}
    except:
        return {}

df_raw['raw_content'] = df_raw['raw_content'].apply(fix_json_tail)
df_raw['parsed_json'] = df_raw['raw_content'].apply(safe_json_parse)

df_json = pd.json_normalize(df_raw['parsed_json']).add_prefix('json_')
df_raw = pd.concat([df_raw.drop(columns=['raw_content','parsed_json']), df_json], axis=1)

print("After JSON flattening:", df_raw.shape)
df_raw.head()

After JSON flattening: (15, 20)


,id,comm_type,source_id,ingested_at,processed_at,is_processed,subject,json_id,json_title,json_duration,json_speakers,json_audio_url,json_video_url,json_dateString,json_host_email,json_calendar_id,json_participants,json_transcript_url,json_organizer_email,json_meeting_attendees
0,de819d74-9b7d-4b60-863c-5f7f634a6402,call,32A19D011A4C41F09FE3A2FB,2025-06-12T13:29:09,2025-03-22T11:37:50,True,Profit-focused composite time-frame,DDCD444CF2A04CA1BA108EE5,Unleash Efficient Convergence,185.486610,"[{'name': 'Eric Ponce'}, {'name': 'Tiffany Hog...",http://www.nelson-guerra.com//audio.mp3,None,2025-04-26T03:21:33,brookegutierrez@yahoo.com,E4D15128947B4EC3B32C965E9805A371,"[E_Ponce@hayes.com, TiffHogan@yahoo.com, lyoun...",http://www.bryant.biz/,alexis81@gonzalez.com,"[{'name': None, 'email': 'E_Ponce@hayes.com', ..."
1,d4746716-3350-4e7f-952f-b39a8dc430d3,call,485F87C4F6E34F3189D1E3E9,2025-03-11T19:29:43,2025-03-06T01:09:40,True,Ergonomic dedicated process improvement,E462A7000DCA4A688FB880B3,Expedite Ubiquitous Synergies,154.750881,"[{'name': 'Kathleen Wiley'}, {'name': 'Thomas ...",https://douglas.biz//audio.mp3,None,2025-05-04T13:03:14,carrie07@brown.com,3F0AFD226DB84ADBA5A68349FB48A162,"[Kathleen_W@yahoo.com, Thom_Heath@hotmail.com,...",https://schneider-villarreal.info/,gordonlaura@barry-alexander.com,"[{'name': None, 'email': 'Kathleen_W@yahoo.com..."
2,e509a9f2-c9a0-4ad0-9489-e22825e40488,meeting,B7E98770F4A542729D65AF41,2025-03-27T06:41:29,2025-01-12T01:51:05,True,Focused clear-thinking parallelism,66DE26DE41BC462AAE1BA7C8,Synthesize Front-End E-Business,234.617079,"[{'name': 'Robert Mckee'}, {'name': 'Austin Ya...",http://anderson.net//audio.mp3,None,2025-05-28T11:39:36,jessicarobinson@hotmail.com,D2C2A12F0FC9426882D15518501CC620,"[timothywaters@hotmail.com, troy51@frey-wilson...",https://www.lopez-carney.com/,thomasstephanie@hotmail.com,"[{'name': None, 'email': 'danielstevens@gmail...."
3,77fb5908-4181-45b8-b484-f7bd4dd294c9,meeting,7A00DCB707B74395B2DD552F,2025-05-06T02:23:25,2025-03-04T16:29:27,True,Re-engineered object-oriented emulation,69D47DC9F39A4AA29B34AEFD,Strategize End-To-End Users,226.883206,"[{'name': 'Jennifer Esparza'}, {'name': 'Chris...",https://www.crane.com//audio.mp3,None,2025-03-11T03:06:20,quinnmark@miller.net,7871794C5B8D43DCA9A22AC592FC7EE0,"[collinscynthia@yahoo.com, gfowler@gmail.com, ...",https://copeland.biz/,katie08@park-brady.net,"[{'name': None, 'email': 'leblancdonald@simmon..."
4,6ba3fa0e-41c1-4ed6-a782-900cc6af3580,call,1C96310A145B445799924A8F,2025-05-09T05:48:27,2025-05-07T15:35:37,True,Focused 24hour policy,0D1BABD93ED04F099F2BCE2D,Syndicate Plug-And-Play Roi,182.514850,"[{'name': 'Mary Velazquez'}, {'name': 'Deborah...",http://www.myers-larson.com//audio.mp3,None,2025-01-13T08:23:52,kristenlee@lewis.com,6B9A54A11475481B9C9825324E53FDA3,"[M.Velazquez@kemp.com, Deb_.Baker@gmail.com, C...",https://www.scott.com/,xschultz@hotmail.com,"[{'name': None, 'email': 'ruizdylan@hotmail.co..."


In [3]:
# Clean and Prepare Columns

df_raw.columns = (df_raw.columns.str.strip().str.lower()
                  .str.replace(' ', '_', regex=False)
                  .str.replace('-', '_', regex=False))


df_raw['meeting_timestamp'] = pd.to_datetime(df_raw['json_datestring'], errors='coerce')
df_raw['meeting_date'] = df_raw['meeting_timestamp'].dt.date

df_raw['meeting_duration'] = pd.to_numeric(df_raw['json_duration'], errors='coerce')


print(df_raw[['meeting_timestamp','meeting_duration', 'meeting_date']].head())

    meeting_timestamp  meeting_duration meeting_date
0 2025-04-26 03:21:33        185.486610   2025-04-26
1 2025-05-04 13:03:14        154.750881   2025-05-04
2 2025-05-28 11:39:36        234.617079   2025-05-28
3 2025-03-11 03:06:20        226.883206   2025-03-11
4 2025-01-13 08:23:52        182.514850   2025-01-13


In [4]:
# 1. dim_comm_type
dim_comm_type = df_raw[['comm_type']].dropna().drop_duplicates().reset_index(drop=True)
dim_comm_type['comm_type_id'] = dim_comm_type.index + 1

# 2. dim_subject
dim_subject = df_raw[['subject']].dropna().drop_duplicates().reset_index(drop=True)
dim_subject['subject_id'] = dim_subject.index + 1

# 3. dim_calendar
dim_calendar = df_raw[['json_calendar_id']].dropna().drop_duplicates().reset_index(drop=True)
dim_calendar = dim_calendar.rename(columns={'json_calendar_id': 'raw_calendar_id'})
dim_calendar['calendar_id'] = dim_calendar.index + 1

# 4. dim_audio
dim_audio = df_raw[['json_audio_url']].dropna().drop_duplicates().reset_index(drop=True)
dim_audio = dim_audio.rename(columns={'json_audio_url': 'raw_audio_url'})
dim_audio['audio_id'] = dim_audio.index + 1

# 5. dim_video
dim_video = df_raw[['json_video_url']].dropna().drop_duplicates().reset_index(drop=True)
dim_video = dim_video.rename(columns={'json_video_url': 'raw_video_url'})
dim_video['video_id'] = dim_video.index + 1

# 6. dim_transcript
dim_transcript = df_raw[['json_transcript_url']].dropna().drop_duplicates().reset_index(drop=True)
dim_transcript = dim_transcript.rename(columns={'json_transcript_url': 'raw_transcript_url'})
dim_transcript['transcript_id'] = dim_transcript.index + 1

# 7. dim_user

# A) From attendees
attendees = (
    pd.json_normalize(df_raw['json_meeting_attendees'].explode().dropna()).fillna('')
)
attendees['email'] = attendees['email'].str.lower()
attendees = attendees[attendees['email'].str.strip() != '']
dim_user = attendees[['name', 'email', 'location', 'displayName', 'phoneNumber']].fillna('')

# B) Add organizer emails
organizer_emails = df_raw['json_organizer_email'].dropna().str.lower().str.strip().unique()
extra_emails = set(organizer_emails) - set(dim_user['email'].unique())
if extra_emails:
    extra_df = pd.DataFrame({
        'name': [''] * len(extra_emails),
        'email': list(extra_emails),
        'location': '',
        'displayName': '',
        'phoneNumber': ''
    })
    dim_user = pd.concat([dim_user, extra_df], ignore_index=True)

# C) Match speaker names to existing emails
all_emails = dim_user['email'].unique().tolist()

def match_name_to_email(name, email_list):
    if not name or not isinstance(name, str): 
        return None
    parts = name.strip().lower().split()
    if len(parts) < 2: 
        return None
    first, last = parts[0], parts[-1]
    f = first[0]
    variants = [
        f"{first}{last}", f"{first}_{last}", f"{first}.{last}",
        f"{f}{last}", f"{f}_{last}", f"{f}.{last}",
        f"{last}{first}", f"{last}_{first}", f"{last}.{first}",
        f"{last}{f}", f"{last}_{f}", f"{last}.{f}",
        f"{first}", f"{last}"
    ]
    for email in email_list:
        local = email.split('@')[0].lower()
        if any(v in local for v in variants): 
            return email
    for email in email_list:
        local = email.split('@')[0].lower()
        if first in local and last in local: 
            return email
    for email in email_list:
        local = email.split('@')[0].lower()
        if first in local or last in local: 
            return email
    return None

speaker_names = pd.json_normalize(df_raw['json_speakers'].explode().dropna())['name'].dropna().unique()
matched, unmatched = [], []

for name in speaker_names:
    email = match_name_to_email(name, all_emails)
    if email:
        matched.append({'name': name, 'email': email})
    else:
        unmatched.append({'name': name, 'email': ''})

matched_df = pd.DataFrame(matched).drop_duplicates(subset='email')
dim_user = pd.merge(dim_user, matched_df, on='email', how='left', suffixes=('', '_spk'))
dim_user['name'] = dim_user.apply(
    lambda x: x['name_spk'] if x['name']=='' and pd.notna(x['name_spk']) else x['name'],
    axis=1
)
dim_user = dim_user.drop(columns=['name_spk'])

# D) Add unmatched speakers
unmatched_df = pd.DataFrame(unmatched)
unmatched_df[['location','displayName','phoneNumber']] = ''
dim_user = pd.concat([dim_user, unmatched_df], ignore_index=True).fillna('')

# E) Finalize dim_user
dim_user = dim_user.drop_duplicates(subset=['email','name']).reset_index(drop=True)
dim_user['user_id'] = dim_user.index + 1

# Preview
print("Dimension tables ready:")
display(dim_comm_type.head(2))
display(dim_subject.head(2))
display(dim_user.head(2))
display(dim_calendar.head(2))
display(dim_audio.head(2))
display(dim_video.head(2))
display(dim_transcript.head(2))

Dimension tables ready:


,comm_type,comm_type_id
0,call,1
1,meeting,2


,subject,subject_id
0,Profit-focused composite time-frame,1
1,Ergonomic dedicated process improvement,2


,name,email,location,displayName,phoneNumber,user_id
0,Eric Ponce,e_ponce@hayes.com,,,,1
1,Tiffany Hogan,tiffhogan@yahoo.com,,,,2


,raw_calendar_id,calendar_id
0,E4D15128947B4EC3B32C965E9805A371,1
1,3F0AFD226DB84ADBA5A68349FB48A162,2


,raw_audio_url,audio_id
0,http://www.nelson-guerra.com//audio.mp3,1
1,https://douglas.biz//audio.mp3,2


,raw_video_url,video_id


,raw_transcript_url,transcript_id
0,http://www.bryant.biz/,1
1,https://schneider-villarreal.info/,2


In [5]:
# Build fact_communication (correct ID mapping)

fact_df = df_raw.copy()

# Merge all dimension foreign keys
fact_df = fact_df.merge(dim_comm_type, on='comm_type', how='left')
fact_df = fact_df.merge(dim_subject, on='subject', how='left')
fact_df = fact_df.merge(dim_calendar, left_on='json_calendar_id', right_on='raw_calendar_id', how='left')
fact_df = fact_df.merge(dim_audio, left_on='json_audio_url', right_on='raw_audio_url', how='left')
fact_df = fact_df.merge(dim_video, left_on='json_video_url', right_on='raw_video_url', how='left')
fact_df = fact_df.merge(dim_transcript, left_on='json_transcript_url', right_on='raw_transcript_url', how='left')

# Assign comm_id from top-level raw id (df_raw['id']), and json inner id -> raw_id
fact_communication = fact_df[[
    'id',                         # top-level id = comm_id
    'json_id',                    # JSON nested meeting ID
    'source_id',
    'comm_type_id',
    'subject_id',
    'calendar_id',
    'audio_id',
    'video_id',
    'transcript_id',
    'ingested_at',
    'processed_at',
    'is_processed',
    'json_title',
    'json_duration'
]].rename(columns={
    'id': 'comm_id',
    'json_id': 'raw_id',
    'json_title': 'raw_title',
    'json_duration': 'raw_duration'
})

# Add datetime_id = same as comm_id (1:1 assumption)
fact_communication['datetime_id'] = fact_communication['comm_id']

print("fact_communication table ready:")
display(fact_communication.head(5))

fact_communication table ready:


,comm_id,raw_id,source_id,comm_type_id,subject_id,calendar_id,audio_id,video_id,transcript_id,ingested_at,processed_at,is_processed,raw_title,raw_duration,datetime_id
0,de819d74-9b7d-4b60-863c-5f7f634a6402,DDCD444CF2A04CA1BA108EE5,32A19D011A4C41F09FE3A2FB,1,1,1,1,NaN,1,2025-06-12T13:29:09,2025-03-22T11:37:50,True,Unleash Efficient Convergence,185.486610,de819d74-9b7d-4b60-863c-5f7f634a6402
1,d4746716-3350-4e7f-952f-b39a8dc430d3,E462A7000DCA4A688FB880B3,485F87C4F6E34F3189D1E3E9,1,2,2,2,NaN,2,2025-03-11T19:29:43,2025-03-06T01:09:40,True,Expedite Ubiquitous Synergies,154.750881,d4746716-3350-4e7f-952f-b39a8dc430d3
2,e509a9f2-c9a0-4ad0-9489-e22825e40488,66DE26DE41BC462AAE1BA7C8,B7E98770F4A542729D65AF41,2,3,3,3,NaN,3,2025-03-27T06:41:29,2025-01-12T01:51:05,True,Synthesize Front-End E-Business,234.617079,e509a9f2-c9a0-4ad0-9489-e22825e40488
3,77fb5908-4181-45b8-b484-f7bd4dd294c9,69D47DC9F39A4AA29B34AEFD,7A00DCB707B74395B2DD552F,2,4,4,4,NaN,4,2025-05-06T02:23:25,2025-03-04T16:29:27,True,Strategize End-To-End Users,226.883206,77fb5908-4181-45b8-b484-f7bd4dd294c9
4,6ba3fa0e-41c1-4ed6-a782-900cc6af3580,0D1BABD93ED04F099F2BCE2D,1C96310A145B445799924A8F,1,5,5,5,NaN,5,2025-05-09T05:48:27,2025-05-07T15:35:37,True,Syndicate Plug-And-Play Roi,182.514850,6ba3fa0e-41c1-4ed6-a782-900cc6af3580


In [6]:
from collections import defaultdict

# 1. Create mappings
email_to_userid = dim_user.set_index(dim_user['email'].str.lower())['user_id'].to_dict()
name_to_userid = (
    dim_user[dim_user['name'].notna() & (dim_user['name'].str.strip() != '')]
    .assign(name_lower=lambda df: df['name'].str.lower())
    .set_index('name_lower')['user_id']
    .to_dict()
)

# 2. Priority role assignment: highest priority kept
priority_roles = ['isOrganiser', 'isSpeaker', 'isParticipant', 'isAttendee']
role_map = {}

for idx, row in df_raw.iterrows():
    comm_id = row['id']
    used_users = set()

    # A) Organiser
    organiser_email = row.get('json_organizer_email', '').strip().lower()
    uid = email_to_userid.get(organiser_email)
    if uid and (comm_id, uid) not in role_map:
        role_map[(comm_id, uid)] = {'comm_id': comm_id, 'user_id': uid,
                                    'isOrganiser': True, 'isSpeaker': False,
                                    'isParticipant': False, 'isAttendee': False}
        used_users.add(uid)

    # B) Speakers
    for spk in row.get('json_speakers') or []:
        name = spk.get('name', '').strip().lower()
        uid = name_to_userid.get(name)
        if uid and uid not in used_users:
            role_map[(comm_id, uid)] = {'comm_id': comm_id, 'user_id': uid,
                                        'isOrganiser': False, 'isSpeaker': True,
                                        'isParticipant': False, 'isAttendee': False}
            used_users.add(uid)

    # C) Participants
    for email in row.get('json_participants') or []:
        email = email.strip().lower()
        uid = email_to_userid.get(email)
        if uid and uid not in used_users:
            role_map[(comm_id, uid)] = {'comm_id': comm_id, 'user_id': uid,
                                        'isOrganiser': False, 'isSpeaker': False,
                                        'isParticipant': True, 'isAttendee': False}
            used_users.add(uid)

    # D) Attendees
    for att in row.get('json_meeting_attendees') or []:
        email = att.get('email', '').strip().lower()
        uid = email_to_userid.get(email)
        if uid and uid not in used_users:
            role_map[(comm_id, uid)] = {'comm_id': comm_id, 'user_id': uid,
                                        'isOrganiser': False, 'isSpeaker': False,
                                        'isParticipant': False, 'isAttendee': True}
            used_users.add(uid)

# 3. Convert to DataFrame
bridge_comm_user = pd.DataFrame(role_map.values())
bridge_comm_user = bridge_comm_user.sort_values(['comm_id', 'user_id']).reset_index(drop=True)

print("Final bridge_comm_user (priority-based):")
display(bridge_comm_user.head(15))

Final bridge_comm_user (priority-based):


,comm_id,user_id,isOrganiser,isSpeaker,isParticipant,isAttendee
0,2228a789-8b05-498b-8fcf-59060a3fc3ff,46,False,True,False,False
1,2228a789-8b05-498b-8fcf-59060a3fc3ff,70,False,True,False,False
2,2228a789-8b05-498b-8fcf-59060a3fc3ff,71,False,False,True,False
3,2228a789-8b05-498b-8fcf-59060a3fc3ff,72,False,False,False,True
4,2228a789-8b05-498b-8fcf-59060a3fc3ff,73,False,False,False,True
5,2228a789-8b05-498b-8fcf-59060a3fc3ff,74,False,False,False,True
6,2228a789-8b05-498b-8fcf-59060a3fc3ff,75,False,False,False,True
7,2228a789-8b05-498b-8fcf-59060a3fc3ff,76,False,False,False,True
8,2228a789-8b05-498b-8fcf-59060a3fc3ff,77,False,False,False,True
9,2228a789-8b05-498b-8fcf-59060a3fc3ff,157,True,False,False,False


In [7]:
# Export to Excel
output_file='output/star_schema_output.xlsx'
with pd.ExcelWriter(output_file,engine='openpyxl') as w:
    dim_comm_type.to_excel(excel_writer=w,sheet_name='dim_comm_type',index=False)
    dim_subject.to_excel(excel_writer=w,sheet_name='dim_subject',index=False)
    dim_user.to_excel(excel_writer=w,sheet_name='dim_user',index=False)
    dim_calendar.to_excel(excel_writer=w,sheet_name='dim_calendar',index=False)
    dim_audio.to_excel(excel_writer=w,sheet_name='dim_audio',index=False)
    dim_video.to_excel(excel_writer=w,sheet_name='dim_video',index=False)
    dim_transcript.to_excel(excel_writer=w,sheet_name='dim_transcript',index=False)
    fact_communication.to_excel(excel_writer=w,sheet_name='fact_communication',index=False)
    bridge_comm_user.to_excel(excel_writer=w,sheet_name='bridge_comm_user',index=False)
print("Export complete:", output_file)

OSError: Cannot save file into a non-existent directory: 'output'